In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
Data = pd.read_csv("C:\\Users\\91828\\Downloads\\Cafe.csv")

Data["Transaction Date"].head(20)
Data["Transaction Date"] = pd.to_datetime(
    Data["Transaction Date"],
    dayfirst= True,
    errors = "coerce"
)

Data["Transaction Date"].head(20)
Data["Transaction Date"].dtype

In [ ]:
Data["Day Name"] = Data["Transaction Date"].dt.day_name()


In [ ]:
Data["Is_Weekend"] = Data["Day Name"].isin(["Saturday", "Sunday"]).astype(int)
Data["Month"] = Data["Transaction Date"].dt.month
Data["Day"] = Data["Transaction Date"].dt.day

In [ ]:
Data["Is_Weekend"] = Data["Day Name"].isin(["Saturday", "Sunday"]).astype(int)
Data["Is_Weekend"].value_counts()

In [ ]:
Data["Price Per Unit"] = pd.to_numeric(Data["Price Per Unit"], errors = "coerce")
Data["Quantity"] = pd.to_numeric(Data["Quantity"], errors = "coerce")

In [ ]:
#clean total spent column 
Data["Total Spent"]= pd.to_numeric(Data["Total Spent"], errors="coerce")

Data["Total Spent"] = Data["Total Spent"].fillna(Data["Price Per Unit"]* Data["Quantity"])

In [ ]:
Data["Price Per Unit"].dtype
Data["Quantity"].dtype
Data["Total Spent"].dtype

In [ ]:
#confirm
Data["Price Per Unit"].head(10).to_list()

In [ ]:
Data["Quantity"].head(10).to_list()

In [ ]:
#Fixing inconsistent spelling 
Data["Payment Method"] = (
    Data["Payment Method"]
    .astype(str)
    .str.strip()
    .str.title()
)
Data["Payment Method"] = Data["Payment Method"].replace({
    "Nan" : "Unknown"
})

In [ ]:
Data["Location"] = Data["Location"].astype(str).str.strip().str.title()

Data["Location"] = Data["Location"].replace({
    "Unknown":"Unknown",
    "In-Store" : "In-Store",
    "Instore" : "In-Store",
    "Takeaway": "Takeaway",
    "Take Away" : "Takeaway"
})

In [ ]:
#filling missing values
Data["Location"].fillna("Unknown", inplace=True)
Data["Item"].fillna("Unknown Item", inplace= True)
Data["Payment Method"].fillna("Unknown", inplace= True)

In [ ]:
#fixing duplicate transaction ID
Data["Transaction ID"].duplicated().sum()


In [ ]:
#Validate numeric columns
Data[["Price Per Unit" , "Quantity", "Total Spent"]].isna().sum()

In [ ]:
Data["Quantity"].fillna(Data["Quantity"].median(),inplace= True)
Data["Price Per Unit"].fillna(Data["Price Per Unit"].median(), inplace= True)

In [ ]:
Data.to_csv("clean_cafe_sales.csv", index= False)

Data Visualization

In [ ]:
#Top 5 Selling Item
Data.groupby("Item")["Quantity"].sum().sort_values(ascending = False).head(5).plot(
    kind="bar",
    figsize= (8,4)
)
plt.title("Top 5 Selling Item")
plt.xlabel("Item")
plt.ylabel("Total Quantity Sold")
plt.show()

Conclusion:
The cafe's top five selling items are Coffee, Juice, Salad, Cake and Sandwich,  with coffee leading by small margin. The quantities sold for these items are very close, showing that customer demand is evenly distributed across the menu.

In [ ]:
#Revenue Generated by Each Item
(Data.groupby("Item")["Total Spent"].sum()
     .sort_values(ascending=False)
     .plot(kind="bar", figsize=(8,4)))

plt.title("Revenue Generated by Each Item")
plt.xlabel("Item")
plt.ylabel("Total Revenue")
plt.show()

Conclusion: 
Salad generates the highest revenue for the cafe, followed by Sandwich and Smoothie. These items significantly outperform the rest of the menu, indicating they are key contributors to overall earnings. 

In [ ]:
#payment method distribution
Data["Payment Method"].value_counts().plot(
    kind="pie",
    autopct="%1.1f%%",
    figsize= (6,6)
)
plt.title("Payment Method Usage")
plt.ylabel("")
plt.show()

Conclusion:
Payment methods are fairly evenly distributed across Credit Card, Cash, and Digital Wallet, each contributing around 22 - 23 percent of transactions.However, a large portion(about 29 percent) is marked as "Unknown", indicating missing or unrecorded payment data. The "Error" category is very small and has little impack on overall analysis.


In [ ]:
#Sales by Location
Data.groupby("Location")["Total Spent"].sum().plot(
    kind= "bar",
    figsize= (8,4)
)
plt.title("Total Sales by Location")
plt.xlabel("Location")
plt.ylabel("Revenue")
plt.show()

Conclusion:
In-store and takeaway orfers generate the highest revenue, with in-store sales slightly leading. A large portion of revenue is also associated with missing or unclassified locations, which suggests inconsistent or incomplete data enery.

In [ ]:
#Weekday vs Weekend Sales
Data.groupby("Is_Weekend")["Quantity"].sum().plot(
    kind= "bar",
    figsize=(6,4)
)
plt.xticks([0,1], ["Weekday", "Weekend"])
plt.title("Total Quantity Sold: Weekday vs Weekend")
plt.xlabel("")
plt.ylabel("Quantity")
plt.show()

Conclusion:
Sales are significantly higher on weekdays compared to weekends,with weekday transactions contributing nearly three times more quantity. This suggests that the ccafe experiences much heavier customer traffic during the week, possible due to workday routines, nearby offices, or regular commuters.

In [ ]:
#Monthly Sales Trend
Data.groupby("Month")["Total Spent"].sum().plot(
    kind= "line",
    marker= "o",
    figsize=(8,4)
)
plt.title("Monthly Sales Trend")
plt.xlabel("Month")
plt.ylabel("Revenue")
plt.show()

In [ ]:
#Sales by Day of Week
Data.groupby("Day Name")["Total Spent"].sum().reindex(
    ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
).plot(kind= "bar", figsize= (10,4))

plt.title("Sales by Day of Week")
plt.xlabel("Day")
plt.ylabel("Revenue")
plt.show()

Conclusion:
Monthly revenue stays relatively stable throughout the year, with no extreme spikes or drops. The highest sales occur around June and October, while February sees the lowest. Overall, the café maintains a consistent performance across months, suggesting steady customer demand year-round.

In [ ]:
#Heatmap: Item vs Payment Method
pivot = Data.pivot_table(
    values= "Total Spent",
    index="Item",
    columns="Payment Method",
    aggfunc="sum"
)
plt.figure(figsize=(10,6))
sns.heatmap(pivot, annot=True, fmt=".0f", cmap="YlGnBu")
plt.title("Heatmap: Item vs Payment Method")
plt.show()

Conclusion:
Salad and Sandwich generate the highest revenue across almost all payment methods, making them the café’s most consistently purchased items. Smoothies and Juice also perform strongly across all methods. Lower-demand items like Cookie, Tea, and the “Unknown” categories show weaker performance. Overall, customer purchasing behavior appears balanced across payment methods, with no single method dominating item-specific revenue.